In [1]:
import pandas as pd
import numpy as np
import torch
from tqdm.auto import tqdm, trange

In [2]:
kpis = pd.read_csv('hackathon_kpis_anonymised.csv')
relative_distances = pd.read_csv('relative_distance.csv')

In [3]:
kpis

,cell_name,timestamp,ho_failure_rate,num_voice_attempts,voice_drop_rate,num_data_attempts,voice_setup_failure_rate,voice_tot_failure_rate,avail_period_duration,bandwidth,throughput_rate,data_setup_failure_rate,data_drop_rate,data_tot_failure_rate,unavail_total_rate,unavail_unplan_rate
0,00_11Z,2020-04-09 15:00:00+00:00,0.333333,0.000927,0.0,0.004527,0.0,0.0,1.0,0.49975,0.000195,0.000000,0.000731,0.000731,0.333364,0.0
1,00_11Z,2020-04-22 14:00:00+00:00,0.360000,0.017609,0.0,0.012312,0.0,0.0,1.0,0.49975,0.000197,0.001075,0.000269,0.000403,0.333364,0.0
2,00_11Z,2020-05-08 21:00:00+00:00,0.333333,0.002780,0.0,0.008115,0.0,0.0,1.0,0.49975,0.000196,0.000000,0.000000,0.000000,0.333364,0.0
3,00_11Z,2020-05-10 13:00:00+00:00,0.380952,0.012048,0.0,0.004898,0.0,0.0,1.0,0.49975,0.000116,0.000000,0.002027,0.002027,0.333364,0.0
4,00_11Z,2020-05-12 03:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.49975,NaN,NaN,NaN,NaN,0.666728,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1042683,23_31W,2020-06-30 23:00:00+00:00,0.333333,0.000000,NaN,0.004197,NaN,NaN,1.0,1.00000,0.000293,0.000000,0.000000,0.000000,0.333364,0.0
1042684,23_31W,2020-07-05 15:00:00+00:00,0.336824,0.004634,0.0,0.011107,0.0,0.0,1.0,1.00000,0.000443,0.000000,0.001639,0.001639,0.333364,0.0
1042685,23_31W,2020-08-12 17:00:00+00:00,0.336585,0.014829,0.0,0.011742,0.0,0.0,1.0,1.00000,0.000352,0.000000,0.000986,0.000986,0.333364,0.0
1042686,23_31W,2020-08-28 14:00:00+00:00,0.333333,0.021316,0.0,0.020003,0.0,0.0,1.0,1.00000,0.000188,0.000662,0.000000,0.000083,0.333364,0.0


In [4]:
kpis.describe()

,ho_failure_rate,num_voice_attempts,voice_drop_rate,num_data_attempts,voice_setup_failure_rate,voice_tot_failure_rate,avail_period_duration,bandwidth,throughput_rate,data_setup_failure_rate,data_drop_rate,data_tot_failure_rate,unavail_total_rate,unavail_unplan_rate
count,469309.000000,912315.000000,630066.000000,912210.000000,630066.000000,630066.000000,1.042688e+06,1.042688e+06,811369.000000,895247.000000,895247.000000,895247.000000,1.042688e+06,1.031935e+06
mean,0.340013,0.012314,0.000720,0.020732,0.000980,0.001046,9.990099e-01,3.435080e-01,0.000180,0.002646,0.000333,0.000664,3.579869e-01,1.435138e-05
std,0.042657,0.027938,0.010291,0.037992,0.018921,0.012102,2.345380e-02,3.452026e-01,0.001994,0.019858,0.002291,0.003553,8.600321e-02,1.056791e-03
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00
25%,0.333333,0.000000,0.000000,0.002528,0.000000,0.000000,1.000000e+00,9.954977e-02,0.000014,0.000000,0.000000,0.000000,3.333642e-01,0.000000e+00
50%,0.333333,0.002780,0.000000,0.007744,0.000000,0.000000,1.000000e+00,9.954977e-02,0.000099,0.000000,0.000000,0.000067,3.333642e-01,0.000000e+00
75%,0.333333,0.011121,0.000000,0.022664,0.000000,0.000000,1.000000e+00,4.997499e-01,0.000248,0.001343,0.000226,0.000576,3.333642e-01,0.000000e+00
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000e+00,1.000000e+00,1.000000,1.000000,1.000000,1.000000,1.000000e+00,1.000000e+00


In [5]:
relative_distances.describe()

,00_11Q,00_11R,00_11V,00_11W,00_11X,00_11Y,00_11Z,00_12Q,00_12W,00_21Q,...,26_22Q,26_31Q,26_32Q,27_11Q,27_12Q,28_11Y,28_21Y,28_31Y,29_11Q,30_11R
count,403.000000,403.000000,403.000000,403.000000,403.000000,403.000000,403.000000,403.000000,403.000000,403.000000,...,403.000000,403.000000,403.000000,403.000000,403.000000,403.000000,403.000000,403.000000,403.000000,403.000000
mean,0.439546,0.439546,0.439546,0.439546,0.439546,0.439546,0.439546,0.439546,0.439546,0.439546,...,0.467032,0.467032,0.467032,0.442353,0.442353,0.612391,0.612391,0.612391,0.609184,0.613234
std,0.288902,0.288902,0.288902,0.288902,0.288902,0.288902,0.288902,0.288902,0.288902,0.288902,...,0.210062,0.210062,0.210062,0.262298,0.262298,0.276762,0.276762,0.276762,0.277313,0.279310
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.204851,0.204851,0.204851,0.204851,0.204851,0.204851,0.204851,0.204851,0.204851,0.204851,...,0.320270,0.320270,0.320270,0.225966,0.225966,0.453367,0.453367,0.453367,0.447092,0.459504
50%,0.370608,0.370608,0.370608,0.370608,0.370608,0.370608,0.370608,0.370608,0.370608,0.370608,...,0.413962,0.413962,0.413962,0.369262,0.369262,0.687191,0.687191,0.687191,0.694825,0.698828
75%,0.624606,0.624606,0.624606,0.624606,0.624606,0.624606,0.624606,0.624606,0.624606,0.624606,...,0.599542,0.599542,0.599542,0.554387,0.554387,0.828659,0.828659,0.828659,0.827446,0.831926
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [6]:
cols = kpis.columns
features = list(cols)
features.remove('cell_name')
features.remove('timestamp')
features

['ho_failure_rate',
 'num_voice_attempts',
 'voice_drop_rate',
 'num_data_attempts',
 'voice_setup_failure_rate',
 'voice_tot_failure_rate',
 'avail_period_duration',
 'bandwidth',
 'throughput_rate',
 'data_setup_failure_rate',
 'data_drop_rate',
 'data_tot_failure_rate',
 'unavail_total_rate',
 'unavail_unplan_rate']

In [7]:
kpis['timestamp'] = pd.to_datetime(kpis['timestamp'])

In [8]:
cells = kpis.groupby('cell_name').groups
cells = {
    cell_name: kpis.loc[idx].sort_values('timestamp').set_index('timestamp')
    for cell_name, idx in cells.items()
}

In [9]:
len(cells)

405

In [10]:
for df in cells.values():
    df.fillna(df.median(), inplace=True)    
    df.fillna(0, inplace=True)

In [11]:
dropped = 0
for key, df in tqdm(cells.items()):
    if (df.isna().sum().sum()):
        dropped+=1
        print(key, len(df), df.isna().sum().sum(), max(df.isna().sum().to_numpy()), sep='\t')
        #print(df.describe())
dropped

  0%|          | 0/405 [00:00<?, ?it/s]

0

In [12]:
SAMPLE_STEPS = 10

def make_samples(df):
    df = df[features]
    n_steps, n_features = df.shape
    nda = df.to_numpy()
    X = np.zeros((n_steps - SAMPLE_STEPS, SAMPLE_STEPS, n_features))
    Y = np.zeros((n_steps - SAMPLE_STEPS, n_features))
    for i in range(SAMPLE_STEPS, n_steps):
        X[i-SAMPLE_STEPS,:,:] = nda[i-SAMPLE_STEPS:i]
        Y[i-SAMPLE_STEPS,:] = nda[i]
    return X, Y

#make_sample(cells['00_11Z'])

In [13]:
all_samples = [
    make_samples(cells[cell_name])
    for cell_name in cells
]

In [14]:
X, Y = map(np.concatenate, zip(*all_samples))

In [15]:
X.shape

(1038638, 10, 14)

In [16]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.25, random_state=42, shuffle=False)

# Model

Try DeepAnT model

In [17]:
class DeepAnT(torch.nn.Module):
    """
        Model : Class for DeepAnT model
    """
    def __init__(self, time_steps, n_features):
        super(DeepAnT, self).__init__()
        self.conv1d_1_layer = torch.nn.Conv1d(in_channels=time_steps, out_channels=16, kernel_size=3)
        self.relu_1_layer = torch.nn.ReLU()
        self.maxpooling_1_layer = torch.nn.MaxPool1d(kernel_size=2)
        self.conv1d_2_layer = torch.nn.Conv1d(in_channels=16, out_channels=16, kernel_size=3)
        self.relu_2_layer = torch.nn.ReLU()
        self.maxpooling_2_layer = torch.nn.MaxPool1d(kernel_size=2)
        self.flatten_layer = torch.nn.Flatten()
        self.dense_1_layer = torch.nn.Linear(32, 16)
        self.relu_3_layer = torch.nn.ReLU()
        self.dropout_layer = torch.nn.Dropout(p=0.25)
        self.dense_2_layer = torch.nn.Linear(16, n_features)
        
    def forward(self, x):
        x = self.conv1d_1_layer(x)
        x = self.relu_1_layer(x)
        x = self.maxpooling_1_layer(x)
        x = self.conv1d_2_layer(x)
        x = self.relu_2_layer(x)
        x = self.maxpooling_2_layer(x)
        x = self.flatten_layer(x)
        x = self.dense_1_layer(x)
        x = self.relu_3_layer(x)
        x = self.dropout_layer(x)
        return self.dense_2_layer(x)

In [29]:
DEVICE = torch.device('cpu')
model = DeepAnT(SAMPLE_STEPS, len(features)).to(DEVICE)
criterion = torch.nn.MSELoss(reduction='mean')
optimizer = torch.optim.Adam(list(model.parameters()), lr=1e-5)

In [30]:
train_data = torch.utils.data.TensorDataset(
    torch.from_numpy(X_train.astype(np.float32)),
    torch.from_numpy(Y_train.astype(np.float32))
)
train_loader = torch.utils.data.DataLoader(dataset=train_data, batch_size=32, shuffle=True)

In [32]:
for epoch in trange(10,11):
    model.train()    
    loss_sum = 0.0
    steps = 0
    for x, y in tqdm(train_loader):
        x, y = x.to(DEVICE), y.to(DEVICE)
        yhat = model(x)
        if torch.any(yhat != yhat):
            print(x)
            print(y)
            print(yhat)
            input()
        loss = criterion(y, yhat)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        loss_sum += loss.item()
        steps += 1
        if steps % 10_000 == 0:
            print('Train loss: {0} @ epoch {1}, step {2}'.format(float(loss_sum/steps), epoch, steps))
    train_loss = loss_sum / steps
    model.eval()
    yhat = model.cpu()(torch.from_numpy(X_test.astype(np.float32)))
    eval_loss = criterion(torch.from_numpy(Y_test.astype(np.float32)), yhat).item()
    print('Epoch {0} done\t train_loss={1} \t eval_loss={2}'.format(epoch, train_loss, eval_loss))
    torch.save(model.state_dict(), 'models/epoch_{0}__eval_loss_{1}.pth'.format(epoch, eval_loss))

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/24344 [00:00<?, ?it/s]

Train loss: 0.0012242997735083918 @ epoch 10, step 10000
Train loss: 0.0012211953995931253 @ epoch 10, step 20000
Epoch 10 done	 train_loss=0.0012177311531268585 	 eval_loss=0.0008903010166250169


In [33]:
model.load_state_dict(torch.load('models/epoch_8__eval_loss_0.0008937545935623348.pth'))

<All keys matched successfully>